<a href="https://colab.research.google.com/github/touchvarit/Data-Analyst-Portfolio/blob/main/Final_Portfolio_Optimization_Project_G_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import cvxpy as cp
from scipy import sparse

In [ ]:
def pull_data(Ticker_names, startDate, endDate, interval):
    """The pulling data function from Yahoo finance.

    Args:
        Ticker_names (list): List of ticker names in Yahoo finance that you want to pull.
        startDate (datetime): Starting date of data you want to pull in datetime form.
        endDate (datetime): Ending date of data you want to pull in datetime form.
        interval (str): Data interval. Valid intervals are:
                       “1m”, “2m”, “5m”, “15m”, “30m”, “60m”, “90m”, “1h”, “1d”, “5d”, “1wk”, “1mo”, “3mo”

    Returns:
        DataFrame: Close_price_dataframe
    """
    # Create dictionaries to store ticker objects and historical data
    Ticker = dict()
    Ticker_historical_data = dict()

    # Loop through ticker names to get ticker objects
    for ticker_name in Ticker_names:
        Ticker[ticker_name] = yf.Ticker(ticker_name)  # Get ticker object

    # Loop through ticker names to get historical data
    for ticker_name in Ticker_names:
        # Get historical data for each ticker
        Ticker_historical_data[ticker_name] = Ticker[ticker_name].history(
            start=startDate, end=endDate, interval=interval
        )[['Close']]
        # Rename the 'Close' column to the ticker name
        Ticker_historical_data[ticker_name].rename(columns={'Close': '%s' % ticker_name}, inplace=True)
        # Set the index to the date
        Ticker_historical_data[ticker_name].index = Ticker_historical_data[ticker_name].index.date

    # Concatenate the historical data for all tickers and drop rows with missing values
    return pd.concat(Ticker_historical_data.values(), axis=1).dropna()

def Log_return(Price_dataframe):
    """Compute the log return from an asset price dataframe.

    Args:
        Price_dataframe (DataFrame): Column: ticker name, Index: date

    Returns:
        DataFrame: Log_return_dataframe
    """
    # Create a shifted DataFrame by shifting Price_dataframe by one period
    dummy = Price_dataframe.shift(periods=1).T

    # Set the first column of the shifted DataFrame to the first row of Price_dataframe
    dummy[Price_dataframe.index[0]] = Price_dataframe.iloc[0].tolist()

    # Create a DataFrame for log returns by taking the natural logarithm of the ratio
    # of Price_dataframe to the shifted DataFrame (to calculate daily log returns)
    return pd.DataFrame(
        np.log(Price_dataframe.values.T / dummy.values),
        columns=Price_dataframe.index.tolist(),
        index=Price_dataframe.columns.tolist()
    ).T

def Mean_Cov(Log_return_dataframe, shifting_day):
    """Compute the Mean and Covariance of each asset.

    Args:
        Log_return_dataframe (DataFrame): Column: ticker name, Index: date
        shifting_day (int): The number of days to shift.

    Returns:
        (Mean, Cov): Mean list, Covariance matrix
    """
    # Compute the mean of log returns for each asset and multiply by the shifting days
    Mean = np.asarray(np.mean(Log_return_dataframe.values.T, axis=1)) * shifting_day

    # Compute the covariance matrix of log returns and multiply by the shifting days
    Cov = np.asmatrix(np.cov(Log_return_dataframe.values.T)) * shifting_day

    return Mean, Cov

def Mean_Variance_optimization(return_array, covariance_matrix, require_return, short_sell=None):
    """
    Optimize the portfolio for mean-variance using convex optimization.

    Args:
        return_array (array): Array of expected returns for each asset.
        covariance_matrix (array): Covariance matrix of asset returns.
        require_return (float): Required return for the portfolio.
        short_sell (str, optional): If short selling is allowed, input 'yes'. Defaults to None.

    Returns:
        tuple: Portfolio weights, objective value, and portfolio return.
    """
    # Define the decision variable for portfolio weights
    X = cp.Variable(len(return_array))

    # Formulate the objective function as the quadratic form of portfolio weights and covariance matrix
    objective_formulation = cp.quad_form(X, covariance_matrix)

    # Define the objective function to minimize the variance
    objective_function = cp.Minimize(objective_formulation)

    # Calculate the expected return of the portfolio
    Return = return_array.T @ X

    # Define constraints based on the specified conditions (short selling or not)
    if short_sell == None:

        constraints = [sum(X) == 1, Return >= require_return, X >= 0]
    else:

        constraints = [sum(X) == 1, Return >= require_return]

    constraints = constraints + [X <= np.array([0.1 for i in range(len(return_array))])]

    # Formulate the convex optimization problem
    problem = cp.Problem(objective_function, constraints)

    # Solve the optimization problem
    problem.solve()

    if problem.status != 'optimal':

        optimal_portfolio = np.array([0 for i in range(len(return_array))])

    else:

        optimal_portfolio = X.value

    # Return the optimized portfolio weights, objective value (variance), and the expected return of the portfolio
    return optimal_portfolio, problem.value, Return.value

def checking_day_trade(Date, Close_price_dataframe):
    """
    Check if the given date is a trading day. If not, find the next trading day.

    Args:
        Date (datetime): The input date to check.
        Close_price_dataframe (DataFrame): DataFrame with trading day indices.

    Returns:
        datetime: The next trading day date.
    """
    # Continue searching for the next trading day until a trading day is found
    while not (Date.date() in Close_price_dataframe.index):
        # Increment the date by one day
        Date = Date + timedelta(days=1)

    # Return the next trading day date
    return Date

def Backtesting_Mean_Variance(len_of_trianing_data, trading_day, period, number_of_trading, require_return, Close_price_dataframe, short_sell = None):
    """
    Backtest a Mean-Variance portfolio strategy over multiple trading periods.

    Args:
        len_of_trianing_data (int): Length of the training data period (in months).
        trading_day (datetime): Initial trading day to start the backtesting.
        period (int): Number of months for each trading period.
        number_of_trading (int): Number of trading periods to backtest.
        require_return (float): Required return for the portfolio in each trading period.
        Close_price_dataframe (DataFrame): DataFrame with close prices for each trading day.

    Returns:
        dict, dict: Returns and portfolio proportions for each trading period.
    """
    trading_day = checking_day_trade(trading_day, Close_price_dataframe)
    start_dict = dict()
    end_dict = dict()
    trianing_data = dict()
    day_trading = dict()
    Mean_dict = dict()
    Cov_dict = dict()
    portfolio_proportions_dict = dict()

    # Loop through each trading period for backtesting
    for i in range(number_of_trading + 1):
        start_dict[i] = (trading_day - len_of_trianing_data * timedelta(days=30)).date()
        end_dict[i] = (trading_day - timedelta(days=1)).date()
        trianing_data[i] = Close_price_dataframe.loc[start_dict[0]:end_dict[i]]
        day_trading[i] = trading_day
        Mean_dict[i], Cov_dict[i] = Mean_Cov(Log_return(trianing_data[i]), period * 30)
        portfolio_proportions_dict[i], __, __ = Mean_Variance_optimization(Mean_dict[i], Cov_dict[i], require_return, short_sell=short_sell)

        trading_day = checking_day_trade(trading_day + timedelta(days=30 * period), Close_price_dataframe)

    R = dict()
    # Calculate returns for each trading period
    for i in range(number_of_trading):
        R[i + 1] = sum(portfolio_proportions_dict[i] * np.log(np.array(Close_price_dataframe.loc[day_trading[i + 1].date()].tolist()) /
                                                       np.array(Close_price_dataframe.loc[day_trading[i].date()].tolist())))

    return R, portfolio_proportions_dict, day_trading, trianing_data

def Backtesting_Mean_Variance(len_of_trianing_data, trading_day, period, number_of_trading, require_return, Close_price_dataframe, short_sell = None):
    """
    Backtest a Mean-Variance portfolio strategy over multiple trading periods.

    Args:
        len_of_trianing_data (int): Length of the training data period (in months).
        trading_day (datetime): Initial trading day to start the backtesting.
        period (int): Number of months for each trading period.
        number_of_trading (int): Number of trading periods to backtest.
        require_return (float): Required return for the portfolio in each trading period.
        Close_price_dataframe (DataFrame): DataFrame with close prices for each trading day.

    Returns:
        dict, dict: Returns and portfolio proportions for each trading period.
    """
    trading_day = checking_day_trade(trading_day, Close_price_dataframe)
    start_dict = dict()
    end_dict = dict()
    trianing_data = dict()
    day_trading = dict()
    Mean_dict = dict()
    Cov_dict = dict()
    portfolio_proportions_dict = dict()

    # Loop through each trading period for backtesting
    for i in range(number_of_trading + 1):
        start_dict[i] = Close_price_dataframe.index[0]
        end_dict[i] = (trading_day - timedelta(days=1)).date()
        trianing_data[i] = Close_price_dataframe.loc[start_dict[i]:end_dict[i]]
        day_trading[i] = trading_day
        Mean_dict[i], Cov_dict[i] = Mean_Cov(Log_return(trianing_data[i]), period * 30)
        portfolio_proportions_dict[i], __, __ = Mean_Variance_optimization(Mean_dict[i], Cov_dict[i], require_return, short_sell=short_sell)

        trading_day = checking_day_trade(trading_day + timedelta(days=30 * period), Close_price_dataframe)

    R = dict()
    # Calculate returns for each trading period
    for i in range(number_of_trading):
        R[i + 1] = sum(portfolio_proportions_dict[i] * np.log(np.array(Close_price_dataframe.loc[day_trading[i + 1].date()].tolist()) /
                                                       np.array(Close_price_dataframe.loc[day_trading[i].date()].tolist())))

    return R, portfolio_proportions_dict, day_trading, trianing_data

def Multivariate_simulation(scenario, Mean, Cov):
    """Simulate the expected return of each asset.

    Args:
        scenario (int): Number of scenarios to simulate.
        Mean (1-D array): Mean list.
        Cov (n-D array): Covariance matrix.

    Returns:
        n-D array: Matrix of scenarios of the expected return of each asset.
    """
    number_of_assets = len(Mean)

    # Generate random samples from a multivariate normal distribution
    mean_simulation = np.random.multivariate_normal(np.zeros(number_of_assets), Cov, size=scenario)

    # Shift the simulated values by the mean to obtain scenarios of expected returns
    return mean_simulation + Mean

### **ทางกลุ่มได้เลือกหุ้น25 ตัวแรกใน S&P500 มาเป็น universe ในการลงทุนเนื่องจากเป็นหุ้นขนาดใหญ่มี liquidity risk น้อยและมีการกระจายตัวของอุตสาหกรรมพอสมควร  เทรดแบบ long only เพื่อให้ระบบจัดการง่าย**


In [ ]:
Ticker_names = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'META', 'TSLA', 'NVDA','AVGO','JPM','UNH','LLY','V','XOM','JNJ','HD','MA','PG','COST','ADBE','ABBV','MRK','CVX','CRM','BRK-B']
startDate = datetime(2005, 1, 1)
endDate = datetime(2018, 1, 1)
interval = '1d'

# Call the pull_data function to retrieve historical close prices for the specified tickers and date range
Close_price_dataframe = pull_data(Ticker_names, startDate, endDate, interval)
Log_return_datafrme = Log_return(Close_price_dataframe)

<ipython-input-2-05ffe7b58228>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ticker_historical_data[ticker_name].rename(columns={'Close': '%s' % ticker_name}, inplace=True)


In [ ]:
shifting_day = 360

Mean, Cov = Mean_Cov(Log_return_datafrme, shifting_day)
print('Mean array is',Mean)
print('Covariance matrix is',Cov)

Mean array is [0.2486277  0.36122431 0.43292196 0.30543386 0.30488326 0.52638627
 0.6220184  0.79816442 0.61481454 0.28561943 0.42243634 0.19504621
 0.31846406 0.02912047 0.23424659 0.34299643 0.32006016 0.1251893
 0.21005838 0.43453423 0.34026682 0.13437831 0.09153109 0.24901545
 0.21577978]
Covariance matrix is [[0.08176837 0.02635037 0.02545848 0.02363271 0.02375568 0.0316126
  0.02895877 0.0324518  0.04575994 0.02106267 0.01678292 0.01191563
  0.02010579 0.01317179 0.01062719 0.01560028 0.02252633 0.01044953
  0.01217295 0.02230541 0.01542109 0.01161926 0.0157261  0.02571191
  0.015172  ]
 [0.02635037 0.07129555 0.03582562 0.03331344 0.03424098 0.03142218
  0.03186617 0.03726671 0.03686043 0.0260796  0.01908008 0.0169333
  0.0269297  0.0168211  0.01489371 0.01906898 0.02850206 0.01475255
  0.01554123 0.03297319 0.02476803 0.015932   0.01981186 0.03454291
  0.01918214]
 [0.02545848 0.03582562 0.11929474 0.04836682 0.04842877 0.0571129
  0.05417583 0.03690834 0.04171678 0.02447944 0.

## **ผลตอบแทนคาดหวัง ที่ทางกลุ่มตั้งไว้คือ 12% ต่อปี ซึ่งนำมาจากขอบบนของ ค่าผลตอบแทนเฉลี่ยที่ดีของอุตสาหกรรมที่ 8-12% ต่อปี (เมื่อเทียบกับผลตอบแทนเฉลี่ยของ S&P500 ที่average ที่ 7%)**

In [ ]:
require_return_2 = 0.12
optimal_Portfolio2,Risk,Return = Mean_Variance_optimization(Mean, Cov, require_return_2)

## เมื่อนำหุ้น 25 ตัวนี้มาoptimize  weight โดยใช้ข้อมูล 3ปีล่าสุดจะได้ และกำหนดให้ห้ามลงทุนในหุ้นตัวใดตัวหนึ่งเกิน 10%

In [ ]:
hist = go.Figure(go.Bar(x= Ticker_names,
    y=optimal_Portfolio2,text=np.round(optimal_Portfolio2,decimals=2),textposition='auto'))

hist.update_layout(xaxis_title = 'Asset', yaxis_title = 'Proportion', title = 'Portfolio Proportion')
hist.show()

In [ ]:
Simulated_Return = Multivariate_simulation(100000, Mean, Cov)
optimal_Portfolio = optimal_Portfolio2
Simulated_Portfolio_Return = [np.dot(optimal_Portfolio, Simulated_Return[i]) for i in range(len(Simulated_Return))]

In [ ]:
# Define portfolio proportions for each asset in the portfolio
portfolio_proportion = optimal_Portfolio

# Specify the confidence level for Value at Risk (VaR) calculation
confidence_level = 0.95

# Calculate the portfolio payoffs by taking the sum of the element-wise product
# of portfolio_proportion and expected_return_simulation for each scenario
payoffs = Simulated_Portfolio_Return

# Calculate the Value at Risk (VaR) at the specified confidence level
VaR = np.percentile(payoffs, (1 - confidence_level) * 100)

# Identify returns below the VaR threshold to calculate Conditional Value at Risk (CVaR)
lower_returns = [i for i in payoffs if i <= VaR]

# Calculate the Conditional Value at Risk (CVaR) by taking the mean of the returns below the VaR threshold
CVaR = -1 * np.mean(lower_returns)

# Print the results
print('The portfolio proportion is ', dict(zip(Ticker_names,portfolio_proportion)))
print('Mean of return is %f' % np.mean(payoffs))
print('VaR is %f' % -VaR)  # Note: Multiplying by -1 to present the VaR as a positive value
print('CVaR is %f' % CVaR)


The portfolio proportion is  {'AAPL': 0.07473477354944782, 'MSFT': 0.007784142020808812, 'AMZN': -1.810848904164583e-24, 'GOOGL': 0.03940971290570274, 'GOOG': 5.191419841479619e-24, 'META': 0.0028273656183290144, 'TSLA': -6.288124711252302e-24, 'NVDA': 0.00472707191663603, 'AVGO': 2.9734997839930785e-24, 'JPM': 6.852518094096679e-24, 'UNH': 0.0768141580250822, 'LLY': 0.07050564263330802, 'V': 0.01011448309083234, 'XOM': 0.1, 'JNJ': 0.1, 'HD': 0.1, 'MA': 4.2547797424055576e-24, 'PG': 0.1, 'COST': 0.1, 'ADBE': 1.86307010280027e-24, 'ABBV': -5.215148753362321e-24, 'MRK': 0.08890440491056405, 'CVX': 0.02417824532928908, 'CRM': 7.13508446365182e-24, 'BRK-B': 0.1}
Mean of return is 0.217617
VaR is -0.000799
CVaR is 0.054506


### **จากนั้นนำ weight ที่ได้มาหา portfolio payoff โดยได้ Mean return ที่ 21% และหาค่า VaR และ CvaR ของ port โดยใช้ข้อมูลจาก ปี 2005-2023**

In [ ]:
# Specify the bin size for the histogram
size = 0.01

# Create a histogram using Plotly (go.Histogram)
hist = go.Figure(data=[go.Histogram(x=payoffs, xbins=dict(
    start=min(payoffs) // size * size,
    end=max(payoffs) // size * size + size,
    size=size
))])

# Update layout settings for the histogram
hist.update_layout(
    title_text='Portfolio payoffs with mean of return are %f' % np.mean(payoffs),
    xaxis_title_text='Return',
    yaxis_title_text='Count',
    bargroupgap=0.1  # gap between bars of the same location coordinates
)

# Identify returns within the VaR range for highlighting
y_high = [i for i in payoffs if (i >= VaR // size * size and i <= VaR // size * size + size)]

# Add a red line representing the VaR threshold
hist.add_shape(
    dict(
        type="line",
        x0=VaR,
        x1=VaR,
        y0=0,
        y1=len(y_high),
        line=dict(
            color="red",  # Color of the line
            width=2  # Line width
        )
    )
)

# Add an arrow annotation pointing at the VaR line
arrow_text = 'VaR is %f' % VaR
hist.add_annotation(
    text=arrow_text,
    x=VaR,
    y=len(y_high),  # Adjust the vertical position of the arrow text
    showarrow=True,
    arrowhead=2,  # Arrowhead style (2: arrowhead at the end of the line)
    arrowwidth=2,  # Width of the arrowhead
    arrowcolor="blue",  # Color of the arrowhead
    ax=-30  # Adjust the x position of the arrow
)

# Show the histogram
hist.show()


In [ ]:
Ticker_names = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'META', 'TSLA', 'NVDA','AVGO','JPM','UNH','LLY','V','XOM','JNJ','HD','MA','PG','COST','ADBE','ABBV','MRK','CVX','CRM','BRK-B']
startDate = datetime(2005, 1, 1)
endDate = datetime(2024, 1, 1)
interval = '1d'

# Call the pull_data function to retrieve historical close prices for the specified tickers and date range
Close_price_dataframe = pull_data(Ticker_names, startDate, endDate, interval)

<ipython-input-2-05ffe7b58228>:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### **เนื่องจากผลการ optimize ทำผลได้ดี จึงทำการทดสอบว่าวิธีการนี้สามารถใช้ได้เมื่อต้องเจอกับข้อมูลที่เกิดขึ้นใหม่ได้หรือไม่ จึงได้นำข้อมูลตั้งแต่ปี 2005-2017 มาทำการ optimize แล้วดูผลลัพท์เมื่อนำไปเทรด ปี 2018 แล้วทำการปรับ port ตาม data ที่ได้มาใหม่ ทุกสิ้นปีจนถึงปี 2023**

In [ ]:
# input variable
len_of_trianing_data = 12 # unit: mouth
trading_day = datetime(2017, 12, 30) # datetime format
period = 12 # unit: mouth
number_of_trading =  5
Close_price_dataframe

,AAPL,MSFT,AMZN,GOOGL,GOOG,META,TSLA,NVDA,AVGO,JPM,...,HD,MA,PG,COST,ADBE,ABBV,MRK,CVX,CRM,BRK-B
2013-01-02,16.769093,22.574516,12.865500,18.099348,18.013729,28.000000,2.357333,2.935979,24.214558,33.084873,...,49.378014,47.807568,50.474308,82.717552,38.340000,22.419832,27.941011,70.240082,42.792500,93.199997
2013-01-03,16.557436,22.272114,12.924000,18.109859,18.024191,27.770000,2.318000,2.938286,24.341064,33.018196,...,49.238010,47.875965,50.154259,83.565506,37.750000,22.234709,28.610132,69.941017,42.177502,93.620003
2013-01-04,16.096230,21.855268,12.957500,18.467718,18.380356,28.760000,2.293333,3.035228,24.184793,33.603443,...,49.144653,47.874088,50.256084,83.296410,38.130001,21.953821,28.366810,70.310043,42.402500,93.849998
2013-01-07,16.001543,21.814407,13.423000,18.387136,18.300158,29.420000,2.289333,2.947519,24.050846,33.640491,...,48.880203,48.703732,49.914200,82.652306,37.939999,21.998505,28.468197,69.832863,42.244999,93.449997
2013-01-08,16.044605,21.699984,13.319000,18.350851,18.264042,29.059999,2.245333,2.882891,23.887131,33.707172,...,49.175774,48.543430,49.834202,82.497398,38.139999,21.519724,28.508747,69.521072,42.492500,93.809998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-18,195.889999,372.649994,154.070007,135.800003,137.190002,344.619995,252.080002,500.769989,1141.750000,166.229996,...,350.809998,424.429993,146.169998,681.239990,599.130005,153.419998,106.040001,149.679993,263.589996,359.649994
2023-12-19,196.940002,373.260010,153.789993,136.649994,138.100006,350.359985,257.220001,496.040009,1139.579956,168.449997,...,352.070007,425.470001,146.169998,677.739990,604.640015,153.600006,106.489998,151.639999,264.339996,361.799988
2023-12-20,194.830002,370.619995,152.119995,138.339996,139.660004,349.279999,247.139999,481.109985,1110.380005,166.550003,...,348.660004,419.450012,143.910004,661.000000,596.059998,151.690002,105.360001,150.250000,260.250000,355.350006
2023-12-21,194.679993,373.540009,153.839996,140.419998,141.800003,354.089996,254.500000,489.899994,1127.290039,167.500000,...,348.970001,423.440002,144.259995,665.159973,600.140015,152.589996,106.389999,150.710007,267.250000,356.140015


### **จากนั้นนำผลลัพธ์ที่ได้จากการทดสอบมาเทียบกับผลตอบแทนจากการถือ S&P500 เฉยๆในช่วงปี 2018-2023**

In [ ]:
Return, portfolio_proportions, day_trading,trianing_data = Backtesting_Mean_Variance(len_of_trianing_data, trading_day, period, number_of_trading, require_return_2, Close_price_dataframe, short_sell = None)

In [ ]:
Ticker_namesM = ['^GSPC']
interval = '1d'
index = [i.date() for i in np.array(list(day_trading.values()))]
# Call the pull_data function to retrieve historical close prices for the specified tickers and date range
SP_500_dataframe = pull_data(Ticker_namesM, startDate, endDate, interval)

# Display the resulting dataframe containing historical close prices
SP_500_dataframe = SP_500_dataframe.loc[index]
log_return_sp_500 = Log_return(SP_500_dataframe)

In [ ]:
pd.DataFrame({'My port':np.array(list(Return.values())),
              'Market':log_return_sp_500.values.T[0][1:]},index = np.array(list(day_trading.values()))[1:])

,My port,Market
2018-12-28,0.033232,-0.081128
2019-12-23,0.245167,0.260056
2020-12-17,0.085953,0.143764
2021-12-13,0.260175,0.226548
2022-12-08,0.068484,-0.163809


In [ ]:
day_trading_l = np.array(list(day_trading.values()))
Return_l = np.array(list(Return.values()))
Return_l = np.exp(Return_l)
log_return_sp_500_l = np.exp(log_return_sp_500.values.T[0])

In [ ]:
W = 100000
W_invest = [W]+[W*np.prod(Return_l[0:i]) for i in range(1,len(Return_l)+1)]
W_invest_Market = [W*np.prod(log_return_sp_500_l[0:i]) for i in range(1,len(log_return_sp_500_l)+1)]

In [ ]:
# Create a scatter plot using Plotly (go.Scatter) for the efficient frontier
fig = go.Figure(data=go.Scatter(x=day_trading_l, y=W_invest, name='Optimal portfolio'))
fig.add_trace(go.Scatter(x=day_trading_l, y=W_invest_Market, name='Market'))

# Update layout settings for the scatter plot
fig.update_layout(
    title_text='The Return of portfolio',  # title of plot
    yaxis_title_text='Wealth',  # y-axis label
    bargroupgap=0.1  # gap between bars of the same location coordinates
)


# Show the plot
fig.show()

# จากนั้นนำ weight ของหุ้นที่เปลี่ยนแปลงไปในแต่ละปีมา plot เพื่อดูการเปลี่ยนแปลงของ weigh ในแต่ละปี

In [ ]:
pd.DataFrame(dict(zip(list(day_trading.values()), np.round(np.array(list(portfolio_proportions.values())), decimals=2))),index=Ticker_names)

,2018-01-02,2018-12-28,2019-12-23,2020-12-17,2021-12-13,2022-12-08
AAPL,0.07,0.07,0.05,0.03,0.02,0.02
MSFT,0.01,0.00,0.00,0.00,-0.00,-0.00
AMZN,0.00,0.00,-0.00,0.07,0.07,0.05
GOOGL,0.04,0.02,0.02,-0.00,0.00,-0.00
GOOG,-0.00,0.00,-0.00,0.03,0.02,0.02
META,0.00,0.01,0.01,0.01,0.01,-0.00
TSLA,-0.00,0.00,-0.00,-0.00,0.00,-0.00
NVDA,0.00,-0.00,-0.00,0.00,-0.00,-0.00
AVGO,0.00,-0.00,-0.00,0.00,0.00,-0.00
JPM,0.00,-0.00,0.00,-0.00,0.00,0.00


In [ ]:
hist = go.Figure(go.Bar(x= Ticker_names,
    y=portfolio_proportions[0], name = '%s'%day_trading[0].date()))

for i in range(1,len(portfolio_proportions)):

    hist.add_trace(go.Bar(
        x=Ticker_names,
        y=portfolio_proportions[i],
        name = '%s'%day_trading[i].date())
        )

hist.update_layout(barmode='overlay')
hist.update_traces(opacity=0.5)

# Show the overlaid histogram
hist.show()
